<a href="https://colab.research.google.com/github/K-SAHASRA/GNN-modelling/blob/main/GNN_model_100_iterations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torch torchvision torchaudio
!pip install torch-geometric
!pip install pandas
!pip install scikit-learn


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [5]:
from google.colab import files

uploaded = files.upload()


KeyboardInterrupt: 

In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import networkx as nx
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import torch.optim as optim
from torch import nn

def preprocess_data(file_path):
    # Load the data
    data = pd.read_csv(file_path, low_memory=False)

    # Identify nodes (microservices and hosts)
    microservices_cpu = [
        'checkoutservice CPU Usage', 'currencyservice CPU Usage', 'emailservice CPU Usage',
        'frontend 1 CPU Usage', 'frontend 2 CPU Usage', 'loadgenerator CPU Usage',
        'paymentservice CPU Usage', 'productcatalogservice CPU Usage',
        'recommendationservice 1 CPU Usage', 'recommendationservice 2 CPU Usage',
        'redis-cart CPU Usage', 'shippingservice CPU Usage'
    ]

    host_nodes_cpu = [
        'Avg Host Node1 CPU Usage', 'Avg Host Node2 CPU Usage',
        'Avg Host Node3 CPU Usage', 'Avg Host Node4 CPU Usage'
    ]

    microservices_mem = [
        'cartservice Mem Usage', 'checkoutservice Mem Usage', 'currencyservice Mem Usage',
        'emailservice Mem Usage', 'frontend 1 Mem Usage', 'frontend 2 Mem Usage',
        'loadgenerator Mem Usage', 'paymentservice Mem Usage',
        'productcatalogservice Mem Usage', 'recommendationservice 1 Mem Usage',
        'recommendationservice 2 Mem Usage', 'redis-cart Mem Usage',
        'shippingservice Mem Usage'
    ]

    host_nodes_mem = [
        'Host Node 1 Mem Usage', 'Host Node 2 Mem Usage',
        'Host Node 3 Mem Usage', 'Host Node 4 Mem Usage'
    ]

    rps_columns = [
        'current_rps GET /', 'current_rps GET /cart',
        'current_rps GET /product/0PUK6V6EV0', 'current_rps GET /product/1YMWWN1N4O',
        'current_rps GET /product/2ZYFJ3GM2N', 'current_rps GET /product/66VCHSJNUP',
        'current_rps GET /product/6E92ZMYYFZ', 'current_rps GET /product/9SIQT8TOJO',
        'current_rps GET /product/L9ECAV7KIM', 'current_rps GET /product/LS4PSXUNUM',
        'current_rps GET /product/OLJCESPC7Z', 'current_rps POST /cart',
        'current_rps POST /cart/checkout', 'current_rps POST /setCurrency',
        'current_rps Aggregated'
    ]

    # Combine all feature columns
    feature_columns = microservices_cpu + host_nodes_cpu + microservices_mem + host_nodes_mem + rps_columns

    # Handle non-numeric entries in numeric columns by converting them to NaN
    for col in feature_columns:
        data[col] = pd.to_numeric(data[col], errors='coerce')

    # Fill missing values in numerical columns with the mean
    data[feature_columns] = data[feature_columns].fillna(data[feature_columns].mean())

    # Ensure no NaN values remain after filling
    data[feature_columns] = data[feature_columns].replace([np.inf, -np.inf], np.nan).fillna(0)

    # Standardize the feature matrix
    scaler = StandardScaler()
    features = scaler.fit_transform(data[feature_columns])

    # Assuming response times are the last columns (or define them explicitly)
    response_time_column = 'Average Response Time'
    response_time = data[response_time_column].values

    # Create adjacency matrix (example: fully connected graph for simplicity)
    num_nodes = len(feature_columns)
    adjacency_matrix = np.ones((num_nodes, num_nodes)) - np.eye(num_nodes)

    # Convert to graph using NetworkX
    G = nx.from_numpy_array(adjacency_matrix)

    # Add features to nodes
    for i, node in enumerate(G.nodes()):
        G.nodes[node]['feature'] = features[i % features.shape[0]]  # Adjust indexing if necessary

    # Convert graph to PyTorch Geometric Data object
    edge_index = torch.tensor(np.array(G.edges).T, dtype=torch.long)
    x = torch.tensor([G.nodes[node]['feature'] for node in G.nodes()], dtype=torch.float)
    y = torch.tensor(response_time, dtype=torch.float)  # Match the shape accordingly

    data = Data(x=x, edge_index=edge_index, y=y)

    return data

# Path to the CSV file
file_path = '/content/100-iterations data combined.csv'
data = preprocess_data(file_path)

# Define the GNN architecture
class GNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = self.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Initialize the model, optimizer, and loss function
input_dim = data.num_features
hidden_dim = 16
output_dim = 1  # Output is the average response time
model = GNN(input_dim, hidden_dim, output_dim)

optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.MSELoss()

# Train the model
def train(data, model, optimizer, loss_fn, epochs=100):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data)
        loss = loss_fn(out, data.y)
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

train(data, model, optimizer, loss_fn)

# List of nodes (microservices and hosts)
nodes = [
    'checkoutservice CPU Usage', 'currencyservice CPU Usage', 'emailservice CPU Usage',
    'frontend 1 CPU Usage', 'frontend 2 CPU Usage', 'loadgenerator CPU Usage',
    'paymentservice CPU Usage', 'productcatalogservice CPU Usage',
    'recommendationservice 1 CPU Usage', 'recommendationservice 2 CPU Usage',
    'redis-cart CPU Usage', 'shippingservice CPU Usage',
    'Avg Host Node1 CPU Usage', 'Avg Host Node2 CPU Usage',
    'Avg Host Node3 CPU Usage', 'Avg Host Node4 CPU Usage',
    'cartservice Mem Usage', 'checkoutservice Mem Usage', 'currencyservice Mem Usage',
    'emailservice Mem Usage', 'frontend 1 Mem Usage', 'frontend 2 Mem Usage',
    'loadgenerator Mem Usage', 'paymentservice Mem Usage',
    'productcatalogservice Mem Usage', 'recommendationservice 1 Mem Usage',
    'recommendationservice 2 Mem Usage', 'redis-cart Mem Usage',
    'shippingservice Mem Usage', 'Host Node 1 Mem Usage', 'Host Node 2 Mem Usage',
    'Host Node 3 Mem Usage', 'Host Node 4 Mem Usage', 'current_rps GET /',
    'current_rps GET /cart', 'current_rps GET /product/0PUK6V6EV0',
    'current_rps GET /product/1YMWWN1N4O', 'current_rps GET /product/2ZYFJ3GM2N',
    'current_rps GET /product/66VCHSJNUP', 'current_rps GET /product/6E92ZMYYFZ',
    'current_rps GET /product/9SIQT8TOJO', 'current_rps GET /product/L9ECAV7KIM',
    'current_rps GET /product/LS4PSXUNUM', 'current_rps GET /product/OLJCESPC7Z',
    'current_rps POST /cart', 'current_rps POST /cart/checkout',
    'current_rps POST /setCurrency', 'current_rps Aggregated'
]

# Evaluate the model and print predicted response times
model.eval()
with torch.no_grad():
    predictions = model(data).squeeze()

# Print the predicted average response times with node names
print("Predicted Average Response Times:")
for node, prediction in zip(nodes, predictions):
    print(f"{node}: {prediction.item()}")


<ipython-input-7-550b33df4719>:89: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  x = torch.tensor([G.nodes[node]['feature'] for node in G.nodes()], dtype=torch.float)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([17892])) that is different to the input size (torch.Size([48, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 0, Loss: 1938.5589599609375
Epoch 10, Loss: 1214.1343994140625
Epoch 20, Loss: 1199.1099853515625
Epoch 30, Loss: 1204.201904296875
Epoch 40, Loss: 1191.7022705078125
Epoch 50, Loss: 1191.403564453125
Epoch 60, Loss: 1191.5445556640625
Epoch 70, Loss: 1191.0511474609375
Epoch 80, Loss: 1190.829833984375
Epoch 90, Loss: 1190.753662109375
Predicted Average Response Times:
checkoutservice CPU Usage: 12.299394607543945
currencyservice CPU Usage: 16.01820945739746
emailservice CPU Usage: 18.380903244018555
frontend 1 CPU Usage: 20.103866577148438
frontend 2 CPU Usage: 21.454877853393555
loadgenerator CPU Usage: 22.55975341796875
paymentservice CPU Usage: 23.487117767333984
productcatalogservice CPU Usage: 24.283714294433594
recommendationservice 1 CPU Usage: 24.981861114501953
recommendationservice 2 CPU Usage: 25.600404739379883
redis-cart CPU Usage: 26.15688705444336
shippingservice CPU Usage: 26.66016960144043
Avg Host Node1 CPU Usage: 27.11805534362793
Avg Host Node2 CPU Usage: 27

In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import networkx as nx
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv

def preprocess_data(file_path):
    # Load the data
    data = pd.read_csv(file_path, low_memory=False)

    # Identify nodes (microservices and hosts)
    microservices_cpu = [
        'checkoutservice CPU Usage', 'currencyservice CPU Usage', 'emailservice CPU Usage',
        'frontend 1 CPU Usage', 'frontend 2 CPU Usage', 'loadgenerator CPU Usage',
        'paymentservice CPU Usage', 'productcatalogservice CPU Usage',
        'recommendationservice 1 CPU Usage', 'recommendationservice 2 CPU Usage',
        'redis-cart CPU Usage', 'shippingservice CPU Usage'
    ]

    host_nodes_cpu = [
        'Avg Host Node1 CPU Usage', 'Avg Host Node2 CPU Usage',
        'Avg Host Node3 CPU Usage', 'Avg Host Node4 CPU Usage'
    ]

    microservices_mem = [
        'cartservice Mem Usage', 'checkoutservice Mem Usage', 'currencyservice Mem Usage',
        'emailservice Mem Usage', 'frontend 1 Mem Usage', 'frontend 2 Mem Usage',
        'loadgenerator Mem Usage', 'paymentservice Mem Usage',
        'productcatalogservice Mem Usage', 'recommendationservice 1 Mem Usage',
        'recommendationservice 2 Mem Usage', 'redis-cart Mem Usage',
        'shippingservice Mem Usage'
    ]

    host_nodes_mem = [
        'Host Node 1 Mem Usage', 'Host Node 2 Mem Usage',
        'Host Node 3 Mem Usage', 'Host Node 4 Mem Usage'
    ]

    rps_columns = [
        'current_rps GET /', 'current_rps GET /cart',
        'current_rps GET /product/0PUK6V6EV0', 'current_rps GET /product/1YMWWN1N4O',
        'current_rps GET /product/2ZYFJ3GM2N', 'current_rps GET /product/66VCHSJNUP',
        'current_rps GET /product/6E92ZMYYFZ', 'current_rps GET /product/9SIQT8TOJO',
        'current_rps GET /product/L9ECAV7KIM', 'current_rps GET /product/LS4PSXUNUM',
        'current_rps GET /product/OLJCESPC7Z', 'current_rps POST /cart',
        'current_rps POST /cart/checkout', 'current_rps POST /setCurrency',
        'current_rps Aggregated'
    ]

    # Combine all feature columns
    feature_columns = microservices_cpu + host_nodes_cpu + microservices_mem + host_nodes_mem + rps_columns

    # Handle non-numeric entries in numeric columns by converting them to NaN
    for col in feature_columns:
        data[col] = pd.to_numeric(data[col], errors='coerce')

    # Fill missing values in numerical columns with the mean
    data[feature_columns] = data[feature_columns].fillna(data[feature_columns].mean())

    # Ensure no NaN values remain after filling
    data[feature_columns] = data[feature_columns].replace([np.inf, -np.inf], np.nan).fillna(0)

    # Standardize the feature matrix
    scaler = StandardScaler()
    features = scaler.fit_transform(data[feature_columns])

    # Prepare target variable (Average Response Time)
    response_time = data['Average Response Time'].values

    # Create multiple graph samples
    graph_samples = []
    num_nodes = len(feature_columns)
    adjacency_matrix = np.ones((num_nodes, num_nodes)) - np.eye(num_nodes)

    for i in range(len(data)):
        G = nx.from_numpy_array(adjacency_matrix)
        for j, node in enumerate(G.nodes()):
            G.nodes[node]['feature'] = features[i, j]

        edge_index = torch.tensor(np.array(G.edges).T, dtype=torch.long)
        x = torch.tensor([G.nodes[node]['feature'] for node in G.nodes()], dtype=torch.float)
        y = torch.tensor(response_time[i], dtype=torch.float)
        graph_samples.append(Data(x=x, edge_index=edge_index, y=y))

    return graph_samples

# Path to the CSV file
file_path = '/content/100-iterations data combined.csv'
graph_samples = preprocess_data(file_path)

# Define the GNN architecture
class GNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = self.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Initialize the model, optimizer, and loss function
input_dim = graph_samples[0].num_features
hidden_dim = 16
output_dim = 1
model = GNN(input_dim, hidden_dim, output_dim)

optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.MSELoss()

# Create a DataLoader for batching the graph samples
loader = DataLoader(graph_samples, batch_size=32, shuffle=True)

# Train the model
def train(loader, model, optimizer, loss_fn, epochs=100):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for data in loader:
            optimizer.zero_grad()
            out = model(data)
            graph_prediction = out.mean(dim=0)  # Average predictions across nodes
            loss = loss_fn(graph_prediction, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {total_loss / len(loader)}')

train(loader, model, optimizer, loss_fn)

# List of nodes (microservices and hosts) including additional features (memory and RPS)
nodes = [
    'checkoutservice CPU Usage', 'currencyservice CPU Usage', 'emailservice CPU Usage',
    'frontend 1 CPU Usage', 'frontend 2 CPU Usage', 'loadgenerator CPU Usage',
    'paymentservice CPU Usage', 'productcatalogservice CPU Usage',
    'recommendationservice 1 CPU Usage', 'recommendationservice 2 CPU Usage',
    'redis-cart CPU Usage', 'shippingservice CPU Usage',
    'Avg Host Node1 CPU Usage', 'Avg Host Node2 CPU Usage',
    'Avg Host Node3 CPU Usage', 'Avg Host Node4 CPU Usage',
    'cartservice Mem Usage', 'checkoutservice Mem Usage', 'currencyservice Mem Usage',
    'emailservice Mem Usage', 'frontend 1 Mem Usage', 'frontend 2 Mem Usage',
    'loadgenerator Mem Usage', 'paymentservice Mem Usage',
    'productcatalogservice Mem Usage', 'recommendationservice 1 Mem Usage',
    'recommendationservice 2 Mem Usage', 'redis-cart Mem Usage',
    'shippingservice Mem Usage', 'Host Node 1 Mem Usage', 'Host Node 2 Mem Usage',
    'Host Node 3 Mem Usage', 'Host Node 4 Mem Usage', 'current_rps GET /',
    'current_rps GET /cart', 'current_rps GET /product/0PUK6V6EV0',
    'current_rps GET /product/1YMWWN1N4O', 'current_rps GET /product/2ZYFJ3GM2N',
    'current_rps GET /product/66VCHSJNUP', 'current_rps GET /product/6E92ZMYYFZ',
    'current_rps GET /product/9SIQT8TOJO', 'current_rps GET /product/L9ECAV7KIM',
    'current_rps GET /product/LS4PSXUNUM', 'current_rps GET /product/OLJCESPC7Z',
    'current_rps POST /cart', 'current_rps POST /cart/checkout',
    'current_rps POST /setCurrency', 'current_rps Aggregated'
]

# Evaluate the model and print predicted response times
def evaluate(loader, model):
    model.eval()
    predictions = []
    with torch.no_grad():
        for data in loader:
            out = model(data)
            predictions.append(out.mean(dim=0).squeeze().tolist())  # Average predictions across nodes
    return predictions

predictions = evaluate(loader, model)

# Print the predicted average response times with node names
print("Predicted Average Response Times:")
for i, prediction in enumerate(predictions):
    print(f"Sample {i}: {prediction}")


/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got -2)

In [17]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import networkx as nx
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv

def preprocess_data(file_path):
    # Load the data
    data = pd.read_csv(file_path, low_memory=False)

    # Identify nodes (microservices and hosts)
    microservices_cpu = [
        'checkoutservice CPU Usage', 'currencyservice CPU Usage', 'emailservice CPU Usage',
        'frontend 1 CPU Usage', 'frontend 2 CPU Usage', 'loadgenerator CPU Usage',
        'paymentservice CPU Usage', 'productcatalogservice CPU Usage',
        'recommendationservice 1 CPU Usage', 'recommendationservice 2 CPU Usage',
        'redis-cart CPU Usage', 'shippingservice CPU Usage'
    ]

    host_nodes_cpu = [
        'Avg Host Node1 CPU Usage', 'Avg Host Node2 CPU Usage',
        'Avg Host Node3 CPU Usage', 'Avg Host Node4 CPU Usage'
    ]

    microservices_mem = [
        'cartservice Mem Usage', 'checkoutservice Mem Usage', 'currencyservice Mem Usage',
        'emailservice Mem Usage', 'frontend 1 Mem Usage', 'frontend 2 Mem Usage',
        'loadgenerator Mem Usage', 'paymentservice Mem Usage',
        'productcatalogservice Mem Usage', 'recommendationservice 1 Mem Usage',
        'recommendationservice 2 Mem Usage', 'redis-cart Mem Usage',
        'shippingservice Mem Usage'
    ]

    host_nodes_mem = [
        'Host Node 1 Mem Usage', 'Host Node 2 Mem Usage',
        'Host Node 3 Mem Usage', 'Host Node 4 Mem Usage'
    ]

    rps_columns = [
        'current_rps GET /', 'current_rps GET /cart',
        'current_rps GET /product/0PUK6V6EV0', 'current_rps GET /product/1YMWWN1N4O',
        'current_rps GET /product/2ZYFJ3GM2N', 'current_rps GET /product/66VCHSJNUP',
        'current_rps GET /product/6E92ZMYYFZ', 'current_rps GET /product/9SIQT8TOJO',
        'current_rps GET /product/L9ECAV7KIM', 'current_rps GET /product/LS4PSXUNUM',
        'current_rps GET /product/OLJCESPC7Z', 'current_rps POST /cart',
        'current_rps POST /cart/checkout', 'current_rps POST /setCurrency',
        'current_rps Aggregated'
    ]

    # Combine all feature columns
    feature_columns = microservices_cpu + host_nodes_cpu + microservices_mem + host_nodes_mem + rps_columns

    # Handle non-numeric entries in numeric columns by converting them to NaN
    for col in feature_columns:
        data[col] = pd.to_numeric(data[col], errors='coerce')

    # Fill missing values in numerical columns with the mean
    data[feature_columns] = data[feature_columns].fillna(data[feature_columns].mean())

    # Ensure no NaN values remain after filling
    data[feature_columns] = data[feature_columns].replace([np.inf, -np.inf], np.nan).fillna(0)

    # Standardize the feature matrix
    scaler = StandardScaler()
    features = scaler.fit_transform(data[feature_columns])

    # Prepare target variable (Average Response Time)
    response_time = data['Average Response Time'].values

    # Create multiple graph samples
    graph_samples = []
    num_nodes = len(feature_columns)
    adjacency_matrix = np.ones((num_nodes, num_nodes)) - np.eye(num_nodes)

    for i in range(len(data)):
        G = nx.from_numpy_array(adjacency_matrix)
        for j, node in enumerate(G.nodes()):
            G.nodes[node]['feature'] = features[i, j]

        edge_index = torch.tensor(np.array(G.edges).T, dtype=torch.long)
        x = torch.tensor([G.nodes[node]['feature'] for node in G.nodes()], dtype=torch.float)
        y = torch.tensor([response_time[i]] * num_nodes, dtype=torch.float)
        graph_samples.append(Data(x=x, edge_index=edge_index, y=y))

    return graph_samples

# Path to the CSV file
file_path = '/content/100-iterations data combined.csv'
graph_samples = preprocess_data(file_path)

# Define the GNN architecture
class GNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = self.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Initialize the model, optimizer, and loss function
input_dim = graph_samples[0].num_features
hidden_dim = 16
output_dim = 1
model = GNN(input_dim, hidden_dim, output_dim)

optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.MSELoss()

# Create a DataLoader for batching the graph samples
loader = DataLoader(graph_samples, batch_size=32, shuffle=True)

# Train the model
def train(loader, model, optimizer, loss_fn, epochs=100):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for data in loader:
            optimizer.zero_grad()
            out = model(data)
            loss = loss_fn(out, data.y.view(-1, 1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {total_loss / len(loader)}')

train(loader, model, optimizer, loss_fn)

# List of nodes (microservices and hosts) including additional features (memory and RPS)
nodes = [
    'checkoutservice CPU Usage', 'currencyservice CPU Usage', 'emailservice CPU Usage',
    'frontend 1 CPU Usage', 'frontend 2 CPU Usage', 'loadgenerator CPU Usage',
    'paymentservice CPU Usage', 'productcatalogservice CPU Usage',
    'recommendationservice 1 CPU Usage', 'recommendationservice 2 CPU Usage',
    'redis-cart CPU Usage', 'shippingservice CPU Usage',
    'Avg Host Node1 CPU Usage', 'Avg Host Node2 CPU Usage',
    'Avg Host Node3 CPU Usage', 'Avg Host Node4 CPU Usage',
    'cartservice Mem Usage', 'checkoutservice Mem Usage', 'currencyservice Mem Usage',
    'emailservice Mem Usage', 'frontend 1 Mem Usage', 'frontend 2 Mem Usage',
    'loadgenerator Mem Usage', 'paymentservice Mem Usage',
    'productcatalogservice Mem Usage', 'recommendationservice 1 Mem Usage',
    'recommendationservice 2 Mem Usage', 'redis-cart Mem Usage',
    'shippingservice Mem Usage', 'Host Node 1 Mem Usage', 'Host Node 2 Mem Usage',
    'Host Node 3 Mem Usage', 'Host Node 4 Mem Usage', 'current_rps GET /',
    'current_rps GET /cart', 'current_rps GET /product/0PUK6V6EV0',
    'current_rps GET /product/1YMWWN1N4O', 'current_rps GET /product/2ZYFJ3GM2N',
    'current_rps GET /product/66VCHSJNUP', 'current_rps GET /product/6E92ZMYYFZ',
    'current_rps GET /product/9SIQT8TOJO', 'current_rps GET /product/L9ECAV7KIM',
    'current_rps GET /product/LS4PSXUNUM', 'current_rps GET /product/OLJCESPC7Z',
    'current_rps POST /cart', 'current_rps POST /cart/checkout',
    'current_rps POST /setCurrency', 'current_rps Aggregated'
]

# Evaluate the model and print predicted response times
def evaluate(loader, model):
    model.eval()
    predictions = []
    with torch.no_grad():
        for data in loader:
            out = model(data)
            predictions.append(out.squeeze().tolist())  # Node-level predictions
    return predictions

predictions = evaluate(loader, model)

# Print the predicted response times with node names
print("Predicted Response Times for Each Node:")
for i, prediction in enumerate(predictions):
    print(f"Sample {i}:")
    for node, value in zip(nodes, prediction):
        print(f"{node}: {value}")



/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got -2)

In [1]:
pip install torch torchvision torchaudio torch-geometric pandas scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [13]:
import pandas as pd
import torch
import torch_geometric
from torch_geometric.data import Data
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Load the data
file_path = '/content/100-iterations data combined.csv'
data = pd.read_csv(file_path, low_memory=False)

# Function to convert string values to floats
def convert_string_to_float(val):
    if isinstance(val, str):
        val = val.replace(',', '')
        if 'K' in val:
            return float(val.replace(' K', '')) * 1000
        elif 'M' in val:
            return float(val.replace(' M', '')) * 1000000
        elif 'B' in val:
            return float(val.replace(' B', '')) * 1000000000
        try:
            return float(val)
        except ValueError:
            return val  # Return the original value if it cannot be converted to float
    return val

# Apply conversion function only to object columns that might contain numeric values
numeric_cols = data.columns[data.dtypes == 'object']
data[numeric_cols] = data[numeric_cols].applymap(convert_string_to_float)

# Drop non-numeric columns (like timestamps) and normalize the features
numeric_data = data.select_dtypes(include=[np.number])
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(numeric_data)

# Define nodes (hosts and microservices)
node_features = torch.tensor(normalized_data, dtype=torch.float)

# Create a fully connected graph (manually creating the edge index)
num_nodes = node_features.shape[1]
row, col = torch.meshgrid(torch.arange(num_nodes), torch.arange(num_nodes))
edge_index = torch.stack([row.flatten(), col.flatten()], dim=0)

# Prepare time-series data
time_window = 10  # Number of time steps to consider
node_features_time_series = []
target_time_series = []

# Loop through the data to create sequences
for i in range(len(node_features) - time_window):
    window = node_features[i:i + time_window]
    node_features_time_series.append(window)
    target_time_series.append(node_features[i + time_window][0])  # Assuming response time is at index 0

node_features_time_series = torch.stack(node_features_time_series)
target_time_series = torch.tensor(target_time_series, dtype=torch.float).view(-1, 1)

# Create graph data object
data = Data(x=node_features_time_series, edge_index=edge_index, y=target_time_series)

import torch.nn as nn
from torch_geometric.nn import GCNConv, GATConv

class GNNModel(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GNNModel, self).__init__()
        # Graph Convolutional Layer
        self.conv1 = GCNConv(in_channels, hidden_channels)
        # Attention Layer
        self.att1 = GATConv(hidden_channels, hidden_channels, heads=4, concat=True)
        # Fully connected layer for output
        self.fc1 = nn.Linear(hidden_channels * 4, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        # Graph Convolution
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        # Attention Mechanism
        x = self.att1(x, edge_index)
        x = torch.relu(x)
        # Output Layer
        x = self.fc1(x)
        return x

# Model instantiation
model = GNNModel(in_channels=data.x.size(-1), hidden_channels=32, out_channels=1)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Training the model
def train(model, data, epochs=100):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data)
        out = out.view(-1, 1)  # Ensure output shape matches target
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

train(model, data)


<ipython-input-13-fd5f1cb1af35>:30: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data[numeric_cols] = data[numeric_cols].applymap(convert_string_to_float)


ValueError: could not convert string to float: '1.11il'

In [14]:
import pandas as pd
import torch
import torch_geometric
from torch_geometric.data import Data
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Load the data
file_path = '/content/100-iterations data combined.csv'
data = pd.read_csv(file_path, low_memory=False)

# Function to convert string values to floats
def convert_string_to_float(val):
    if isinstance(val, str):
        val = val.replace(',', '')
        if 'K' in val:
            return float(val.replace(' K', '')) * 1000
        elif 'M' in val:
            return float(val.replace(' M', '')) * 1000000
        elif 'B' in val:
            return float(val.replace(' B', '')) * 1000000000
        try:
            return float(val)
        except ValueError:
            return val  # Return the original value if it cannot be converted to float
    return val

# Apply conversion function only to object columns that might contain numeric values
numeric_cols = data.columns[data.dtypes == 'object']
data[numeric_cols] = data[numeric_cols].applymap(convert_string_to_float)

# Drop non-numeric columns (like timestamps) and normalize the features
numeric_data = data.select_dtypes(include=[np.number])
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(numeric_data)

# Define nodes (hosts and microservices)
node_features = torch.tensor(normalized_data, dtype=torch.float)

# Create a fully connected graph (manually creating the edge index)
num_nodes = node_features.shape[1]
row, col = torch.meshgrid(torch.arange(num_nodes), torch.arange(num_nodes))
edge_index = torch.stack([row.flatten(), col.flatten()], dim=0)

# Prepare time-series data
time_window = 10  # Number of time steps to consider
node_features_time_series = []
target_time_series = []

# Loop through the data to create sequences
for i in range(len(node_features) - time_window):
    window = node_features[i:i + time_window]
    node_features_time_series.append(window)
    target_time_series.append(node_features[i + time_window][0])  # Assuming response time is at index 0

node_features_time_series = torch.stack(node_features_time_series)
target_time_series = torch.tensor(target_time_series, dtype=torch.float).view(-1, 1)

# Create graph data object
data = Data(x=node_features_time_series, edge_index=edge_index, y=target_time_series)

import torch.nn as nn
from torch_geometric.nn import GCNConv, GATConv

class GNNModel(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GNNModel, self).__init__()
        # Graph Convolutional Layer
        self.conv1 = GCNConv(in_channels, hidden_channels)
        # Attention Layer
        self.att1 = GATConv(hidden_channels, hidden_channels, heads=4, concat=True)
        # Fully connected layer for output
        self.fc1 = nn.Linear(hidden_channels * 4, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        # Graph Convolution
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        # Attention Mechanism
        x = self.att1(x, edge_index)
        x = torch.relu(x)
        # Output Layer
        x = self.fc1(x)
        return x

# Model instantiation
model = GNNModel(in_channels=data.x.size(-1), hidden_channels=32, out_channels=1)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Training the model
def train(model, data, epochs=100):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data)
        out = out.view(-1, 1)  # Ensure output shape matches target
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

train(model, data)


<ipython-input-14-fd5f1cb1af35>:30: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data[numeric_cols] = data[numeric_cols].applymap(convert_string_to_float)


ValueError: could not convert string to float: '1.11il'

In [8]:
# Install necessary libraries
!pip install torch torch-geometric scikit-learn

import pandas as pd
import torch
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from torch_geometric.data import Data
from torch_geometric.utils import dense_to_sparse
import torch.nn as nn
from torch_geometric.nn import GCNConv, GATConv

# Load the data
file_path = '/content/100-iterations data combined.csv'
data = pd.read_csv(file_path, low_memory=False)

# Drop the 'Time' column and handle missing values
data_cleaned = data.drop(columns=['Time']).dropna()

# Normalize the numerical columns
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(data_cleaned)

# Convert to torch tensor
node_features = torch.tensor(normalized_data, dtype=torch.float)

# Create a fully connected graph
num_nodes = node_features.shape[1]
adj_matrix = torch.ones((num_nodes, num_nodes)) - torch.eye(num_nodes)
edge_index = dense_to_sparse(adj_matrix)[0]

# Prepare time-series data
time_window = 10  # Number of time steps to consider
node_features_time_series = []
target_time_series = []

# Assuming 'Average Response Time' is the target, located at column index 24
target_index = list(data_cleaned.columns).index('Average Response Time')

# Loop through the data to create sequences
for i in range(len(node_features) - time_window):
    window = node_features[i:i + time_window]
    node_features_time_series.append(window)
    target_time_series.append(node_features[i + time_window][target_index])

node_features_time_series = torch.stack(node_features_time_series)
target_time_series = torch.tensor(target_time_series, dtype=torch.float).view(-1, 1)

# Create graph data object
graph_data = Data(x=node_features_time_series, edge_index=edge_index, y=target_time_series)

# Define the GNN model
class GNNModel(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GNNModel, self).__init__()
        # Graph Convolutional Layer
        self.conv1 = GCNConv(in_channels, hidden_channels)
        # Attention Layer
        self.att1 = GATConv(hidden_channels, hidden_channels, heads=4, concat=True)
        # Fully connected layer for output
        self.fc1 = nn.Linear(hidden_channels * 4, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        # Graph Convolution
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        # Attention Mechanism
        x = self.att1(x, edge_index)
        x = torch.relu(x)
        # Output Layer
        x = self.fc1(x)
        return x

# Model instantiation
model = GNNModel(in_channels=graph_data.x.size(-1), hidden_channels=32, out_channels=1)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Training the model
def train(model, data, epochs=100):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data)
        out = out.view(-1, 1)  # Ensure output shape matches target
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

train(model, graph_data)


ValueError: could not convert string to float: '1 K'

In [9]:
import pandas as pd
import torch
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from torch_geometric.data import Data
from torch_geometric.utils import dense_to_sparse
import torch.nn as nn
from torch_geometric.nn import GCNConv, GATConv

# Load the data
file_path = '/content/100-iterations data combined.csv'
data = pd.read_csv(file_path, low_memory=False)

# Handle non-numeric values like '1 K', '1 M', etc.
def convert_to_numeric(value):
    if isinstance(value, str):
        value = value.replace(' K', 'e3').replace(' M', 'e6')
        try:
            return float(value)
        except ValueError:
            return np.nan
    return value

# Apply the conversion to the entire DataFrame
data_cleaned = data.drop(columns=['Time']).applymap(convert_to_numeric)

# Drop rows with missing values after conversion
data_cleaned = data_cleaned.dropna()

# Normalize the numerical columns
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(data_cleaned)

# Convert to torch tensor
node_features = torch.tensor(normalized_data, dtype=torch.float)

# Create a fully connected graph
num_nodes = node_features.shape[1]
adj_matrix = torch.ones((num_nodes, num_nodes)) - torch.eye(num_nodes)
edge_index = dense_to_sparse(adj_matrix)[0]

# Prepare time-series data
time_window = 10  # Number of time steps to consider
node_features_time_series = []
target_time_series = []

# Assuming 'Average Response Time' is the target, located at column index 24
target_index = list(data_cleaned.columns).index('Average Response Time')

# Loop through the data to create sequences
for i in range(len(node_features) - time_window):
    window = node_features[i:i + time_window]
    node_features_time_series.append(window)
    target_time_series.append(node_features[i + time_window][target_index])

node_features_time_series = torch.stack(node_features_time_series)
target_time_series = torch.tensor(target_time_series, dtype=torch.float).view(-1, 1)

# Create graph data object
graph_data = Data(x=node_features_time_series, edge_index=edge_index, y=target_time_series)

# Define the GNN model
class GNNModel(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GNNModel, self).__init__()
        # Graph Convolutional Layer
        self.conv1 = GCNConv(in_channels, hidden_channels)
        # Attention Layer
        self.att1 = GATConv(hidden_channels, hidden_channels, heads=4, concat=True)
        # Fully connected layer for output
        self.fc1 = nn.Linear(hidden_channels * 4, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        # Graph Convolution
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        # Attention Mechanism
        x = self.att1(x, edge_index)
        x = torch.relu(x)
        # Output Layer
        x = self.fc1(x)
        return x

# Model instantiation
model = GNNModel(in_channels=graph_data.x.size(-1), hidden_channels=32, out_channels=1)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Training the model
def train(model, data, epochs=100):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data)
        out = out.view(-1, 1)  # Ensure output shape matches target
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

train(model, graph_data)


<ipython-input-9-ff935656cd8c>:25: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_cleaned = data.drop(columns=['Time']).applymap(convert_to_numeric)


ValueError: Found array with 0 sample(s) (shape=(0, 66)) while a minimum of 1 is required by MinMaxScaler.

In [10]:
# Load the data
data = pd.read_csv(file_path, low_memory=False)

# Inspect the data
print("Original data:")
print(data.head())

# Check data types
print("\nData types before conversion:")
print(data.dtypes)

# Handle non-numeric entries and attempt conversion
data_cleaned = data.drop(columns=['Time'])

# Convert specific problematic columns if necessary
# Example: Handle columns with units like 'K', 'MB', etc.
def custom_convert(series):
    series = series.str.replace(' K', '000')
    series = series.str.replace(' M', '000000')
    return pd.to_numeric(series, errors='coerce')

# Apply conversion (use custom_convert if needed)
data_cleaned = data_cleaned.apply(lambda col: custom_convert(col) if col.dtype == 'object' else col)

# Print out the cleaned data to inspect
print("Data after conversion:")
print(data_cleaned.head())

# Handle NaNs (either drop or fill)
data_cleaned = data_cleaned.fillna(0)  # Example: Fill NaNs with 0

# Normalize the numerical columns
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(data_cleaned)

print("Data normalization successful.")


Original data:
             Time  Avg Host Node1 CPU Usage  Host Node1 CPU 0 Usage  \
0  6/28/2024 0:00                      24.1                    19.4   
1  6/28/2024 0:00                      23.4                    25.8   
2  6/28/2024 0:00                      23.1                    21.4   
3  6/28/2024 0:00                      23.1                    24.2   
4  6/28/2024 0:00                      23.2                    20.0   

   Host Node1 CPU 1 Usage  Avg Host Node2 CPU Usage  Host Node2 CPU 0 Usage  \
0                    28.8                      20.3                    21.4   
1                    21.0                      19.5                    17.6   
2                    24.8                      19.4                    20.6   
3                    22.0                      19.3                    22.6   
4                    26.4                      20.4                    23.4   

   Host Node2 CPU 1 Usage  Avg Host Node3 CPU Usage  Host Node3 CPU 0 Usage  \
0   

In [7]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116
!pip install torch-geometric


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116


In [11]:
# Load the data
data = pd.read_csv(file_path, low_memory=False)

# Inspect the data
print("Original data:")
print(data.head())

# Check data types
print("\nData types before conversion:")
print(data.dtypes)

# Handle non-numeric entries and attempt conversion
data_cleaned = data.drop(columns=['Time'])

# Convert specific problematic columns if necessary
# Example: Handle columns with units like 'K', 'MB', etc.
def custom_convert(series):
    series = series.str.replace(' K', '000')
    series = series.str.replace(' M', '000000')
    return pd.to_numeric(series, errors='coerce')

# Apply conversion (use custom_convert if needed)
data_cleaned = data_cleaned.apply(lambda col: custom_convert(col) if col.dtype == 'object' else col)

# Print out the cleaned data to inspect
print("Data after conversion:")
print(data_cleaned.head())

# Handle NaNs (either drop or fill)
data_cleaned = data_cleaned.fillna(0)  # Example: Fill NaNs with 0

# Normalize the numerical columns
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(data_cleaned)

print("Data normalization successful.")

node_features = torch.tensor(normalized_data, dtype=torch.float)

# Create a fully connected graph
num_nodes = node_features.shape[1]
adj_matrix = torch.ones((num_nodes, num_nodes)) - torch.eye(num_nodes)
edge_index = dense_to_sparse(adj_matrix)[0]

# Prepare time-series data
time_window = 10  # Number of time steps to consider
node_features_time_series = []
target_time_series = []

# Assuming 'Average Response Time' is the target, located at column index 24
target_index = list(data_cleaned.columns).index('Average Response Time')

# Loop through the data to create sequences
for i in range(len(node_features) - time_window):
    window = node_features[i:i + time_window]
    node_features_time_series.append(window)
    target_time_series.append(node_features[i + time_window][target_index])

node_features_time_series = torch.stack(node_features_time_series)
target_time_series = torch.tensor(target_time_series, dtype=torch.float).view(-1, 1)

# Create graph data object
graph_data = Data(x=node_features_time_series, edge_index=edge_index, y=target_time_series)

Original data:
             Time  Avg Host Node1 CPU Usage  Host Node1 CPU 0 Usage  \
0  6/28/2024 0:00                      24.1                    19.4   
1  6/28/2024 0:00                      23.4                    25.8   
2  6/28/2024 0:00                      23.1                    21.4   
3  6/28/2024 0:00                      23.1                    24.2   
4  6/28/2024 0:00                      23.2                    20.0   

   Host Node1 CPU 1 Usage  Avg Host Node2 CPU Usage  Host Node2 CPU 0 Usage  \
0                    28.8                      20.3                    21.4   
1                    21.0                      19.5                    17.6   
2                    24.8                      19.4                    20.6   
3                    22.0                      19.3                    22.6   
4                    26.4                      20.4                    23.4   

   Host Node2 CPU 1 Usage  Avg Host Node3 CPU Usage  Host Node3 CPU 0 Usage  \
0   

In [12]:
# Normalize the numerical columns
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(data_cleaned)

# Convert to torch tensor
node_features = torch.tensor(normalized_data, dtype=torch.float)

# Prepare time-series data
time_window = 10  # Number of time steps to consider
node_features_time_series = []
target_time_series = []

# Assuming 'Average Response Time' is the target, located at column index 24
target_index = list(data_cleaned.columns).index('Average Response Time')

# Loop through the data to create sequences
for i in range(len(node_features) - time_window):
    window = node_features[i:i + time_window]
    node_features_time_series.append(window)
    target_time_series.append(node_features[i + time_window][target_index])

node_features_time_series = torch.stack(node_features_time_series)
target_time_series = torch.tensor(target_time_series, dtype=torch.float).view(-1, 1)

# Now, create the edge_index for each time step
num_nodes = node_features_time_series.size(1)
# Create a fully connected graph for each time step
edge_index = torch.tensor([[i, j] for i in range(num_nodes) for j in range(num_nodes) if i != j], dtype=torch.long).t().contiguous()

# Replicate edge_index for each time step in the sequence
edge_index = edge_index.repeat(1, node_features_time_series.size(0))

# Flatten node features for the graph
graph_data = Data(x=node_features_time_series.view(-1, node_features_time_series.size(2)), edge_index=edge_index, y=target_time_series)

# Define the GNN model
class GNNModel(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GNNModel, self).__init__()
        # Graph Convolutional Layer
        self.conv1 = GCNConv(in_channels, hidden_channels)
        # Attention Layer
        self.att1 = GATConv(hidden_channels, hidden_channels, heads=4, concat=True)
        # Fully connected layer for output
        self.fc1 = nn.Linear(hidden_channels * 4, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        # Graph Convolution
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        # Attention Mechanism
        x = self.att1(x, edge_index)
        x = torch.relu(x)
        # Output Layer
        x = self.fc1(x)
        # Average across nodes to get a single prediction per time step
        x = torch.mean(x.view(-1, num_nodes, x.size(-1)), dim=1)
        return x

# Model instantiation
model = GNNModel(in_channels=node_features_time_series.size(2), hidden_channels=32, out_channels=1)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Training the model
def train(model, data, epochs=100):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

train(model, graph_data)

Epoch 1, Loss: 0.024103239178657532
Epoch 2, Loss: 0.01763368956744671
Epoch 3, Loss: 0.013651769608259201
Epoch 4, Loss: 0.011805187910795212
Epoch 5, Loss: 0.011553137563169003
Epoch 6, Loss: 0.01207597367465496
Epoch 7, Loss: 0.012491019442677498
Epoch 8, Loss: 0.012363789603114128
Epoch 9, Loss: 0.011754780076444149
Epoch 10, Loss: 0.010935795493423939
Epoch 11, Loss: 0.010178636759519577
Epoch 12, Loss: 0.009657519869506359
Epoch 13, Loss: 0.009427890181541443
Epoch 14, Loss: 0.009445696137845516
Epoch 15, Loss: 0.009600930847227573
Epoch 16, Loss: 0.009763184003531933
Epoch 17, Loss: 0.00983368419110775
Epoch 18, Loss: 0.009776567108929157
Epoch 19, Loss: 0.009613925591111183
Epoch 20, Loss: 0.00940379686653614
Epoch 21, Loss: 0.009210987016558647
Epoch 22, Loss: 0.009081490337848663
Epoch 23, Loss: 0.009029681794345379
Epoch 24, Loss: 0.00903895404189825
Epoch 25, Loss: 0.00907073076814413
Epoch 26, Loss: 0.00908296275883913
Epoch 27, Loss: 0.009047010913491249
Epoch 28, Loss: 0

In [15]:
model.eval()
with torch.no_grad():
    predictions = model(graph_data)

# Print predictions
predictions = predictions.view(-1).tolist()
for i, pred in enumerate(predictions):
    print(f'Time step {i}: Predicted Average Response Time = {pred:.4f}')

Streaming output truncated to the last 5000 lines.
Time step 12882: Predicted Average Response Time = 0.0586
Time step 12883: Predicted Average Response Time = 0.0559
Time step 12884: Predicted Average Response Time = 0.0554
Time step 12885: Predicted Average Response Time = 0.0898
Time step 12886: Predicted Average Response Time = 0.1017
Time step 12887: Predicted Average Response Time = 0.0993
Time step 12888: Predicted Average Response Time = 0.0954
Time step 12889: Predicted Average Response Time = 0.0956
Time step 12890: Predicted Average Response Time = 0.0993
Time step 12891: Predicted Average Response Time = 0.0984
Time step 12892: Predicted Average Response Time = 0.0967
Time step 12893: Predicted Average Response Time = 0.0982
Time step 12894: Predicted Average Response Time = 0.0958
Time step 12895: Predicted Average Response Time = 0.0568
Time step 12896: Predicted Average Response Time = 0.0429
Time step 12897: Predicted Average Response Time = 0.0398
Time step 12898: Pred

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([319765924, 1])) that is different to the input size (torch.Size([17882, 10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (10) must match the size of tensor b (319765924) at non-singleton dimension 1

In [13]:
# Load the data
data = pd.read_csv(file_path, low_memory=False)

# Inspect the data
print("Original data:")
print(data.head())

# Check data types
print("\nData types before conversion:")
print(data.dtypes)

# Handle non-numeric entries and attempt conversion
data_cleaned = data.drop(columns=['Time'])

# Convert specific problematic columns if necessary
# Example: Handle columns with units like 'K', 'MB', etc.
def custom_convert(series):
    series = series.str.replace(' K', '000')
    series = series.str.replace(' M', '000000')
    return pd.to_numeric(series, errors='coerce')

# Apply conversion (use custom_convert if needed)
data_cleaned = data_cleaned.apply(lambda col: custom_convert(col) if col.dtype == 'object' else col)

# Print out the cleaned data to inspect
print("Data after conversion:")
print(data_cleaned.head())

# Handle NaNs (either drop or fill)
data_cleaned = data_cleaned.fillna(0)  # Example: Fill NaNs with 0

# Normalize the numerical columns
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(data_cleaned)

print("Data normalization successful.")

node_features = torch.tensor(normalized_data, dtype=torch.float)

# Create a fully connected graph
num_nodes = node_features.shape[1]
adj_matrix = torch.ones((num_nodes, num_nodes)) - torch.eye(num_nodes)
edge_index = dense_to_sparse(adj_matrix)[0]

# Prepare time-series data
time_window = 10  # Number of time steps to consider
node_features_time_series = []
target_time_series = []

# Assuming 'Average Response Time' is the target, located at column index 24
target_index = list(data_cleaned.columns).index('Average Response Time')

# Loop through the data to create sequences
for i in range(len(node_features) - time_window):
    window = node_features[i:i + time_window]
    node_features_time_series.append(window)
    target_time_series.append(node_features[i + time_window][target_index])

node_features_time_series = torch.stack(node_features_time_series)
target_time_series = torch.tensor(target_time_series, dtype=torch.float).view(-1, 1)

# Create graph data object
graph_data = Data(x=node_features_time_series, edge_index=edge_index, y=target_time_series)


# Normalize the numerical columns
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(data_cleaned)

# Convert to torch tensor
node_features = torch.tensor(normalized_data, dtype=torch.float)

# Prepare time-series data
time_window = 10  # Number of time steps to consider
node_features_time_series = []
target_time_series = []

# Assuming 'Average Response Time' is the target, located at column index 24
target_index = list(data_cleaned.columns).index('Average Response Time')

# Loop through the data to create sequences
for i in range(len(node_features) - time_window):
    window = node_features[i:i + time_window]
    node_features_time_series.append(window)
    target_time_series.append(node_features[i + time_window][target_index])

node_features_time_series = torch.stack(node_features_time_series)
target_time_series = torch.tensor(target_time_series, dtype=torch.float).view(-1, 1)

# Now, create the edge_index for each time step
num_nodes = node_features_time_series.size(1)
# Create a fully connected graph for each time step
edge_index = torch.tensor([[i, j] for i in range(num_nodes) for j in range(num_nodes) if i != j], dtype=torch.long).t().contiguous()

# Replicate edge_index for each time step in the sequence
edge_index = edge_index.repeat(1, node_features_time_series.size(0))

# Flatten node features for the graph
num_time_steps = node_features_time_series.size(0)
graph_data = Data(
    x=node_features_time_series.view(-1, node_features_time_series.size(2)),
    edge_index=edge_index,
    y=target_time_series.repeat(num_time_steps, 1)  # Ensure `y` matches the shape of `x`
)
# Define the GNN model
class GNNModel(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GNNModel, self).__init__()
        # Graph Convolutional Layer
        self.conv1 = GCNConv(in_channels, hidden_channels)
        # Attention Layer
        self.att1 = GATConv(hidden_channels, hidden_channels, heads=4, concat=True)
        # Fully connected layer for output
        self.fc1 = nn.Linear(hidden_channels * 4, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        # Graph Convolution
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        # Attention Mechanism
        x = self.att1(x, edge_index)
        x = torch.relu(x)
        # Output Layer
        x = self.fc1(x)
        # Average across nodes to get a single prediction per time step
        x = torch.mean(x.view(-1, num_nodes, x.size(-1)), dim=1)
        return x

# Model instantiation
model = GNNModel(in_channels=node_features_time_series.size(2), hidden_channels=32, out_channels=1)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Training the model
def train(model, data, epochs=100):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data)
        # Ensure output shape matches target shape
        out = out.view(-1, 1)  # Flatten to match the target
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')
# Make predictions
model.eval()
with torch.no_grad():
    node_predictions = model(graph_data)

# Ensure correct reshaping of predictions
node_predictions = node_predictions.view(num_time_steps, num_nodes, 1)  # Shape: [num_time_steps, num_nodes, 1]

# Aggregate predictions for each node across time steps
average_predictions_per_node = node_predictions.mean(dim=0)  # Shape: [num_nodes, 1]

# Print average response time predictions for each node
for node_idx in range(num_nodes):
    print(f'Node {node_idx}: Average Predicted Response Time = {average_predictions_per_node[node_idx].item():.4f}')


Original data:
             Time  Avg Host Node1 CPU Usage  Host Node1 CPU 0 Usage  \
0  6/28/2024 0:00                      24.1                    19.4   
1  6/28/2024 0:00                      23.4                    25.8   
2  6/28/2024 0:00                      23.1                    21.4   
3  6/28/2024 0:00                      23.1                    24.2   
4  6/28/2024 0:00                      23.2                    20.0   

   Host Node1 CPU 1 Usage  Avg Host Node2 CPU Usage  Host Node2 CPU 0 Usage  \
0                    28.8                      20.3                    21.4   
1                    21.0                      19.5                    17.6   
2                    24.8                      19.4                    20.6   
3                    22.0                      19.3                    22.6   
4                    26.4                      20.4                    23.4   

   Host Node2 CPU 1 Usage  Avg Host Node3 CPU Usage  Host Node3 CPU 0 Usage  \
0   

RuntimeError: shape '[17882, 10, 1]' is invalid for input of size 17882

In [4]:
import torch
import torch.nn as nn
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, GATConv  # Import necessary layers
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Load the data
file_path = '/content/small-dataset.xlsx'
data = pd.read_excel(file_path)

# Drop the 'Time' column
data_cleaned = data.drop(columns=['Time'])

# Convert specific problematic columns if necessary
def custom_convert(series):
    series = series.str.replace(' K', '000')
    series = series.str.replace(' M', '000000')
    return pd.to_numeric(series, errors='coerce')

# Apply conversion (use custom_convert if needed)
data_cleaned = data_cleaned.apply(lambda col: custom_convert(col) if col.dtype == 'object' else col)

# Fill NaNs with 0
data_cleaned = data_cleaned.fillna(0)

# Normalize the numerical columns
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(data_cleaned)

# Convert to torch tensor
node_features = torch.tensor(normalized_data, dtype=torch.float)

# Prepare time-series data
time_window = 10  # Number of time steps to consider
node_features_time_series = []
target_time_series = []

# Loop through the data to create sequences
for i in range(len(node_features) - time_window):
    window = node_features[i:i + time_window]
    node_features_time_series.append(window)
    target_time_series.append(node_features[i + time_window, -1])  # Assume last column is the target (e.g., response time)

node_features_time_series = torch.stack(node_features_time_series)
target_time_series = torch.tensor(target_time_series, dtype=torch.float).view(-1, 1)

# Now, create the edge_index for each time step
num_nodes = node_features_time_series.size(2)  # Since each node is a feature
# Create a fully connected graph for each time step
edge_index = torch.tensor([[i, j] for i in range(num_nodes) for j in range(num_nodes) if i != j], dtype=torch.long).t().contiguous()

# Replicate edge_index for each time step in the sequence
edge_index = edge_index.repeat(1, node_features_time_series.size(0))

# Flatten node features for the graph
num_time_steps = node_features_time_series.size(0)
graph_data = Data(
    x=node_features_time_series.view(-1, num_nodes),
    edge_index=edge_index,
    y=target_time_series
)

class GNNModel(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.att1 = GATConv(hidden_channels, hidden_channels, heads=4, concat=True)
        self.fc1 = nn.Linear(hidden_channels * 4, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # Graph Convolution
        x = self.conv1(x, edge_index)
        x = torch.relu(x)

        # Attention Mechanism
        x = self.att1(x, edge_index)
        x = torch.relu(x)

        # Output Layer
        x = self.fc1(x)

        # Reshape x to have shape (batch_size, num_nodes, output_dim)
        batch_size = data.y.size(0)
        num_nodes = x.size(0) // batch_size
        x = x.view(batch_size, num_nodes, -1)

        # Average across nodes to get a single prediction per time step
        x = torch.mean(x, dim=1)

        return x


# Model instantiation
model = GNNModel(in_channels=node_features_time_series.size(2), hidden_channels=32, out_channels=1)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

#Modified training function
def train(model, data, epochs=100, print_interval=10):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()

        # Print loss every `print_interval` epochs
        if epoch % print_interval == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

# Train the model
train(model, graph_data, epochs=100)


# Make predictions
model.eval()
with torch.no_grad():
    node_predictions = model(graph_data)

# Check the shape of node_predictions before reshaping
node_predictions.shape
# List of service and host node names
service_names = [
    "checkoutservice CPU Usage", "currencyservice CPU Usage", "emailservice CPU Usage",
    "frontend 1 CPU Usage", "frontend 2 CPU Usage", "loadgenerator CPU Usage",
    "paymentservice CPU Usage", "productcatalogservice CPU Usage",
    "recommendationservice 1 CPU Usage", "recommendationservice 2 CPU Usage",
    "redis-cart CPU Usage", "shippingservice CPU Usage"
]

host_node_names = [
    "Avg Host Node1 CPU Usage", "Avg Host Node2 CPU Usage",
    "Avg Host Node3 CPU Usage", "Avg Host Node4 CPU Usage"
]

# Print predicted average response times for services
print("Predicted Average Response Times:")
for i, name in enumerate(service_names):
    print(f"{name}: {node_predictions[i].item()}")

# Print predicted average response times for host nodes
for i, name in enumerate(host_node_names):
    print(f"{name}: {node_predictions[len(service_names) + i].item()}")

# Continue for other metrics such as Memory Usage or RPS if needed
# List of service and host node memory names
service_mem_names = [
    "cartservice Mem Usage", "checkoutservice Mem Usage", "currencyservice Mem Usage",
    "emailservice Mem Usage", "frontend 1 Mem Usage", "frontend 2 Mem Usage",
    "loadgenerator Mem Usage", "paymentservice Mem Usage",
    "productcatalogservice Mem Usage", "recommendationservice 1 Mem Usage",
    "recommendationservice 2 Mem Usage", "redis-cart Mem Usage",
    "shippingservice Mem Usage"
]

host_mem_names = [
    "Host Node 1 Mem Usage", "Host Node 2 Mem Usage",
    "Host Node 3 Mem Usage", "Host Node 4 Mem Usage"
]

# Print predicted memory usage for services
for i, name in enumerate(service_mem_names):
    print(f"{name}: {node_predictions[len(service_names) + len(host_node_names) + i].item()}")

# Print predicted memory usage for host nodes
for i, name in enumerate(host_mem_names):
    print(f"{name}: {node_predictions[len(service_names) + len(host_node_names) + len(service_mem_names) + i].item()}")

# List of RPS metrics
rps_names = [
    "current_rps GET /", "current_rps GET /cart", "current_rps GET /product/0PUK6V6EV0",
    "current_rps GET /product/1YMWWN1N4O", "current_rps GET /product/2ZYFJ3GM2N",
    "current_rps GET /product/66VCHSJNUP", "current_rps GET /product/6E92ZMYYFZ",
    "current_rps GET /product/9SIQT8TOJO", "current_rps GET /product/L9ECAV7KIM",
    "current_rps GET /product/LS4PSXUNUM", "current_rps GET /product/OLJCESPC7Z",
    "current_rps POST /cart", "current_rps POST /cart/checkout",
    "current_rps POST /setCurrency", "current_rps Aggregated"
]

# Print predicted RPS metrics
for i, name in enumerate(rps_names):
    print(f"{name}: {node_predictions[len(service_names) + len(host_node_names) + len(service_mem_names) + len(host_mem_names) + i].item()}")

Epoch 0, Loss: 0.0007070864085108042
Epoch 10, Loss: 0.00019601642270572484
Epoch 20, Loss: 7.054956222418696e-05
Epoch 30, Loss: 2.9753098715445958e-05
Epoch 40, Loss: 1.4120206287770998e-05
Epoch 50, Loss: 5.831424005009467e-06
Epoch 60, Loss: 3.145683194816229e-06
Epoch 70, Loss: 2.635568989717285e-06
Epoch 80, Loss: 1.9516112388373585e-06
Epoch 90, Loss: 1.6986944046948338e-06
Predicted Average Response Times:
checkoutservice CPU Usage: -0.00015180670015979558
currencyservice CPU Usage: -0.00015185978554654866
emailservice CPU Usage: -0.00015134252316784114
frontend 1 CPU Usage: -0.0001516588090453297
frontend 2 CPU Usage: -0.00015157349116634578
loadgenerator CPU Usage: -0.0001521684171166271
paymentservice CPU Usage: -0.0003919662849511951
productcatalogservice CPU Usage: -0.0019302224973216653
recommendationservice 1 CPU Usage: -0.0012259614886716008
recommendationservice 2 CPU Usage: 0.0006834639934822917
redis-cart CPU Usage: -0.0005478348466567695
shippingservice CPU Usage: 0

In [4]:
import torch
import torch.nn as nn
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, GATConv  # Import necessary layers
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import psutil

def print_memory_usage(stage):
    process = psutil.Process()
    memory_usage = process.memory_info().rss / (1024 * 1024)  # Convert to MB
    print(f"[{stage}] Memory Usage: {memory_usage:.2f} MB")

# Load the data
file_path = '/content/small-dataset.xlsx'
data = pd.read_excel(file_path)

# Drop the 'Time' column
data_cleaned = data.drop(columns=['Time'])
print_memory_usage("After Data Loading")

# Convert specific problematic columns if necessary
def custom_convert(series):
    series = series.str.replace(' K', '000')
    series = series.str.replace(' M', '000000')
    return pd.to_numeric(series, errors='coerce')

# Apply conversion (use custom_convert if needed)
data_cleaned = data_cleaned.apply(lambda col: custom_convert(col) if col.dtype == 'object' else col)
data_cleaned = data_cleaned.fillna(0)
print_memory_usage("After Data Cleaning")

# Normalize the numerical columns
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(data_cleaned)
print_memory_usage("After Normalization")

# Convert to torch tensor
node_features = torch.tensor(normalized_data, dtype=torch.float)

# Prepare time-series data
time_window = 5  # Smaller time window to reduce memory usage
node_features_time_series = []
target_time_series = []

# Loop through the data to create sequences
for i in range(len(node_features) - time_window):
    window = node_features[i:i + time_window]
    node_features_time_series.append(window)
    target_time_series.append(node_features[i + time_window, -1])  # Assume last column is the target (e.g., response time)

node_features_time_series = torch.stack(node_features_time_series)
target_time_series = torch.tensor(target_time_series, dtype=torch.float).view(-1, 1)
print_memory_usage("After Preparing Time-Series Data")

# Now, create the edge_index for each time step
num_nodes = node_features_time_series.size(2)  # Since each node is a feature
# Create a fully connected graph for each time step
edge_index = torch.tensor([[i, j] for i in range(num_nodes) for j in range(num_nodes) if i != j], dtype=torch.long).t().contiguous()

# Replicate edge_index for each time step in the sequence
edge_index = edge_index.repeat(1, node_features_time_series.size(0))

# Flatten node features for the graph
num_time_steps = node_features_time_series.size(0)
graph_data = Data(
    x=node_features_time_series.view(-1, num_nodes),
    edge_index=edge_index,
    y=target_time_series
)
print_memory_usage("After Preparing Graph Data")

# Batch the data using DataLoader
batch_size = 32
loader = DataLoader([graph_data], batch_size=batch_size, shuffle=True)

class GNNModel(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.att1 = GATConv(hidden_channels, hidden_channels, heads=4, concat=True)
        self.fc1 = nn.Linear(hidden_channels * 4, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # Graph Convolution
        x = self.conv1(x, edge_index)
        x = torch.relu(x)

        # Attention Mechanism
        x = self.att1(x, edge_index)
        x = torch.relu(x)

        # Output Layer
        x = self.fc1(x)

        # Reshape x to have shape (batch_size, num_nodes, output_dim)
        batch_size = data.y.size(0)
        num_nodes = x.size(0) // batch_size
        x = x.view(batch_size, num_nodes, -1)

        # Average across nodes to get a single prediction per time step
        x = torch.mean(x, dim=1)

        return x


# Model instantiation
model = GNNModel(in_channels=node_features_time_series.size(2), hidden_channels=32, out_channels=1)
print_memory_usage("After Model Instantiation")

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

#Modified training function with batch processing
def train(model, loader, epochs=100, print_interval=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in loader:
            optimizer.zero_grad()
            out = model(batch)
            loss = criterion(out, batch.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        # Print loss every `print_interval` epochs
        if epoch % print_interval == 0:
            print(f'Epoch {epoch}, Loss: {total_loss/len(loader):.4f}')
            print_memory_usage(f"After Epoch {epoch}")

# Train the model
train(model, loader, epochs=100)

# Make predictions
model.eval()
with torch.no_grad():
    node_predictions = model(graph_data)

# Check the shape of node_predictions before reshaping
print(node_predictions.shape)
# List of service and host node names
service_names = [
    "checkoutservice CPU Usage", "currencyservice CPU Usage", "emailservice CPU Usage",
    "frontend 1 CPU Usage", "frontend 2 CPU Usage", "loadgenerator CPU Usage",
    "paymentservice CPU Usage", "productcatalogservice CPU Usage",
    "recommendationservice 1 CPU Usage", "recommendationservice 2 CPU Usage",
    "redis-cart CPU Usage", "shippingservice CPU Usage"
]

host_node_names = [
    "Avg Host Node1 CPU Usage", "Avg Host Node2 CPU Usage",
    "Avg Host Node3 CPU Usage", "Avg Host Node4 CPU Usage"
]

# Print predicted average response times for services
print("Predicted Average Response Times:")
for i, name in enumerate(service_names):
    print(f"{name}: {node_predictions[i].item()}")

# Print predicted average response times for host nodes
for i, name in enumerate(host_node_names):
    print(f"{name}: {node_predictions[len(service_names) + i].item()}")

# Continue for other metrics such as Memory Usage or RPS if needed
# List of service and host node memory names
service_mem_names = [
    "cartservice Mem Usage", "checkoutservice Mem Usage", "currencyservice Mem Usage",
    "emailservice Mem Usage", "frontend 1 Mem Usage", "frontend 2 Mem Usage",
    "loadgenerator Mem Usage", "paymentservice Mem Usage",
    "productcatalogservice Mem Usage", "recommendationservice 1 Mem Usage",
    "recommendationservice 2 Mem Usage", "redis-cart Mem Usage",
    "shippingservice Mem Usage"
]

host_mem_names = [
    "Host Node 1 Mem Usage", "Host Node 2 Mem Usage",
    "Host Node 3 Mem Usage", "Host Node 4 Mem Usage"
]

# Print predicted memory usage for services
for i, name in enumerate(service_mem_names):
    print(f"{name}: {node_predictions[len(service_names) + len(host_node_names) + i].item()}")

# Print predicted memory usage for host nodes
for i, name in enumerate(host_mem_names):
    print(f"{name}: {node_predictions[len(service_names) + len(host_node_names) + len(service_mem_names) + i].item()}")

# List of RPS metrics
rps_names = [
    "current_rps GET /", "current_rps GET /cart", "current_rps GET /product/0PUK6V6EV0",
    "current_rps GET /product/1YMWWN1N4O", "current_rps GET /product/2ZYFJ3GM2N",
    "current_rps GET /product/66VCHSJNUP", "current_rps GET /product/6E92ZMYYFZ",
    "current_rps GET /product/9SIQT8TOJO", "current_rps GET /product/L9ECAV7KIM",
    "current_rps GET /product/LS4PSXUNUM", "current_rps GET /product/OLJCESPC7Z",
    "current_rps POST /cart", "current_rps POST /cart/checkout",
    "current_rps POST /setCurrency", "current_rps Aggregated"
]

# Print predicted RPS metrics
for i, name in enumerate(rps_names):
    print(f"{name}: {node_predictions[len(service_names) + len(host_node_names) + len(service_mem_names) + len(host_mem_names) + i].item()}")


[After Data Loading] Memory Usage: 575.26 MB
[After Data Cleaning] Memory Usage: 576.10 MB
[After Normalization] Memory Usage: 576.10 MB
[After Preparing Time-Series Data] Memory Usage: 579.39 MB
[After Preparing Graph Data] Memory Usage: 587.42 MB


/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


[After Model Instantiation] Memory Usage: 588.43 MB
Epoch 0, Loss: 0.0007
[After Epoch 0] Memory Usage: 680.76 MB
Epoch 10, Loss: 0.0001
[After Epoch 10] Memory Usage: 718.33 MB
Epoch 20, Loss: 0.0001
[After Epoch 20] Memory Usage: 730.79 MB
Epoch 30, Loss: 0.0000
[After Epoch 30] Memory Usage: 718.46 MB
Epoch 40, Loss: 0.0000
[After Epoch 40] Memory Usage: 736.82 MB
Epoch 50, Loss: 0.0000
[After Epoch 50] Memory Usage: 749.14 MB
Epoch 60, Loss: 0.0000
[After Epoch 60] Memory Usage: 742.98 MB
Epoch 70, Loss: 0.0000
[After Epoch 70] Memory Usage: 718.46 MB
Epoch 80, Loss: 0.0000
[After Epoch 80] Memory Usage: 736.82 MB
Epoch 90, Loss: 0.0000
[After Epoch 90] Memory Usage: 736.82 MB
torch.Size([94, 1])
Predicted Average Response Times:
checkoutservice CPU Usage: 8.266828808700666e-05
currencyservice CPU Usage: 8.179638825822622e-05
emailservice CPU Usage: 8.247960067819804e-05
frontend 1 CPU Usage: 8.323210931848735e-05
frontend 2 CPU Usage: 8.291211270261556e-05
loadgenerator CPU Usage: